In [ ]:
!pip install mediapipe
!pip install tensorflow
!pip install kaggle
!kaggle datasets download -d grassknoted/asl-alphabet

In [ ]:
import patoolib
patoolib.extract_archive("asl-alphabet.zip")

In [1]:
import tensorflow as tf
import numpy as np
import mediapipe as mp
import time
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image

print("TF version:", tf.__version__)
cv2.__version__

TF version: 2.8.0


'4.5.5'

In [3]:
path_Train = "C:/HackWesTX/asl_alphabet_train/asl_alphabet_train"
data_dir = Path(path_Train)
batch_size = 100
img_height = 48
img_width = 48

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [6]:
#defining the model

from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

# number of possible label values
nb_classes = 29

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(nb_classes)
])

opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [8]:
model.fit(train_ds,validation_data=val_ds,epochs = 15)

Epoch 1/15
696/696 [==============================] - 121s 174ms/step - loss: 1.2546 - accuracy: 0.6280 - val_loss: 1.1615 - val_accuracy: 0.6548
Epoch 2/15
696/696 [==============================] - 121s 174ms/step - loss: 1.0676 - accuracy: 0.6819 - val_loss: 1.0134 - val_accuracy: 0.6918
Epoch 3/15
696/696 [==============================] - 125s 180ms/step - loss: 0.9277 - accuracy: 0.7221 - val_loss: 0.9135 - val_accuracy: 0.7251
Epoch 4/15
696/696 [==============================] - 131s 188ms/step - loss: 0.8185 - accuracy: 0.7552 - val_loss: 0.7959 - val_accuracy: 0.7589
Epoch 5/15
696/696 [==============================] - 130s 187ms/step - loss: 0.7270 - accuracy: 0.7810 - val_loss: 0.7303 - val_accuracy: 0.7784
Epoch 6/15
696/696 [==============================] - 129s 184ms/step - loss: 0.6527 - accuracy: 0.8032 - val_loss: 0.6481 - val_accuracy: 0.7993
Epoch 7/15
696/696 [==============================] - 129s 185ms/step - loss: 0.5843 - accuracy: 0.8249 - val_loss: 0.5760 -

In [9]:
model.save("asl.model")

INFO:tensorflow:Assets written to: asl.model\assets
